In [1]:
#from google.colab import drive
#drive.mount('/content/drive/')
import multiprocessing
print("num of cpus:", multiprocessing.cpu_count())


num of cpus: 128


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#folder_path = '/content/drive/MyDrive/TDS_data_15000'
folder_path = './15K files/'
#Load and preprocess the data
def load_data(folder_path):
  data = {}
  for filename in os.listdir(folder_path):
      file_path = os.path.join(folder_path, filename)
      if os.path.isfile(file_path):
          with open(file_path, 'r') as file:
              df1 = pd.read_csv(file_path, header=None, names=['des_flux', 'detrap_en', 'def_conc'], index_col=None, squeeze = True)
              #df1 = df1.fillna(df1.mean()) # Replace NaN with column average
              data[filename] = df1
    
              num_datapts = len(df1["des_flux"])
  # Combine all the data into a single dataframe
  df = pd.concat(data.values())

  return df, num_datapts

df, num_datapts = load_data(folder_path)

des_flux=df['des_flux']
des_flux=des_flux.tolist()
num_files=int(len(des_flux)/num_datapts) 
des_flux=np.reshape(des_flux, [num_files, num_datapts])
    # This assumes that all spectra in the dataset/folder have the same
    # amount of datapoints
#display(des_flux)
print(type(des_flux[0]), des_flux.shape)

detrap_en=df['detrap_en']
detrap_en=detrap_en.dropna()
detrap_en=detrap_en.tolist()
num_files=int(len(detrap_en)/4)
detrap_en=np.reshape(detrap_en, [num_files, 4])
#print(detrap_en, detrap_en.shape)

def_conc=df['def_conc']
def_conc=def_conc.dropna()
def_conc=def_conc.tolist()
num_files=int(len(def_conc)/4)
def_conc=np.reshape(def_conc, [num_files, 4])
#print(def_conc, def_conc.shape)

def_param = np.concatenate([detrap_en, def_conc], axis=1)
print(def_param.shape)

<class 'numpy.ndarray'> (15000, 301)
(15000, 8)


##### Manually adding features

In [3]:
# Peak information
# Peak heights, position, and full width at half max
from scipy.signal import peak_widths, find_peaks
from sklearn.metrics import auc

peak_heights = []
fwhm = []
peak_positions = []
area_under_curves = []
stdev = []

for i in des_flux:
    peaks, _ = find_peaks(i)
    peak_pos = np.argmax(i)
    peak_height = np.max(i)
    peak_width, _, _, _ = peak_widths(i, [peak_pos], rel_height=0.5) 
        #taking only the first output (widths)
    peak_heights.append(peak_height)
    fwhm.append(peak_width[0])
    peak_positions.append(peak_pos)
# Area under curve
# Calculated using the trapezoidal method
    area_under_curve = auc(range(0,len(i)), i)
    area_under_curves.append(area_under_curve)
#STDEV
    spectra_std = np.std(i)
    stdev.append(spectra_std)


In [4]:
# PCA Components based on normalized spectra

# Standardizing and applying PCA using a pipeline
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5))
])

scaled_des_flux = pca_pipe.fit_transform(des_flux)
result_pca = pca_pipe.fit_transform(scaled_des_flux)


# Formatting them into columns in a df
pca_col = ['PCA Component {}'.format(i+1) for i in range(result_pca.shape[1])]
pca_df = pd.DataFrame(data=result_pca, columns=pca_col)

display(pca_df)

,PCA Component 1,PCA Component 2,PCA Component 3,PCA Component 4,PCA Component 5
0,-0.099691,-0.283993,2.283271,-0.434680,1.321750
1,-1.023512,0.928424,1.282852,-0.847930,0.413341
2,0.615698,-0.147251,0.422033,-0.646426,0.047908
3,0.559444,-0.731473,0.721612,-0.056484,-0.009878
4,0.382054,0.185622,0.154274,-0.394287,-0.512061
...,...,...,...,...,...
14995,-0.229937,0.336439,1.588797,0.257341,-0.720488
14996,1.118923,0.255319,-1.771784,-1.386127,-0.163215
14997,0.743302,2.775150,-1.227949,0.532333,2.123537
14998,2.359334,3.977421,1.805939,10.054454,8.551973


In [5]:
### Making the features matrix 
features_matrix = pd.DataFrame()
features_matrix['peak_heights'] = peak_heights 
features_matrix['peak_pos'] = peak_positions
features_matrix['fwhm'] = fwhm
features_matrix['auc'] = area_under_curves
features_matrix['std'] = stdev
features_matrix = pd.concat([features_matrix, pca_df],axis=1)
display(features_matrix)

,peak_heights,peak_pos,fwhm,auc,std,PCA Component 1,PCA Component 2,PCA Component 3,PCA Component 4,PCA Component 5
0,1.104600e+19,139,62.925853,7.168618e+20,3.670862e+18,-0.099691,-0.283993,2.283271,-0.434680,1.321750
1,8.952698e+18,108,76.863979,6.980819e+20,3.100856e+18,-1.023512,0.928424,1.282852,-0.847930,0.413341
2,6.449705e+18,135,118.976213,6.585460e+20,2.208307e+18,0.615698,-0.147251,0.422033,-0.646426,0.047908
3,6.255561e+18,150,99.830005,5.768126e+20,2.134721e+18,0.559444,-0.731473,0.721612,-0.056484,-0.009878
4,3.924261e+18,109,155.580504,5.837127e+20,1.634831e+18,0.382054,0.185622,0.154274,-0.394287,-0.512061
...,...,...,...,...,...,...,...,...,...,...
14995,5.871478e+18,121,70.829252,4.089573e+20,1.928194e+18,-0.229937,0.336439,1.588797,0.257341,-0.720488
14996,8.661125e+18,207,75.580094,8.404429e+20,2.822632e+18,1.118923,0.255319,-1.771784,-1.386127,-0.163215
14997,7.093851e+18,95,43.338462,9.985840e+20,2.359050e+18,0.743302,2.775150,-1.227949,0.532333,2.123537
14998,7.871170e+18,242,122.573237,9.833929e+20,2.465215e+18,2.359334,3.977421,1.805939,10.054454,8.551973


### Raw 0.3K/s, 15K dataset (Smoothed)

In [6]:
### Splitting and preparing the raw data

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor


# Split the data into train and test sets
X_train, X_test= train_test_split(des_flux, test_size=0.1, random_state=42)
y_train_raw, y_test_raw = train_test_split(def_param, test_size=0.1, random_state=42)

# Normalize the data using StandardScaler 
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_norm = scaler.fit_transform(np.array(X_train).reshape(-1, num_datapts)) # Change lists into array
X_test_norm = scaler.transform(np.array(X_test).reshape(-1, num_datapts))

X_train_transform_raw = np.reshape(X_train_norm, (X_train_norm.shape[0], -1))
X_test_transform_raw = np.reshape(X_test_norm, (X_test_norm.shape[0], -1))

print(X_train_transform_raw.shape)
print(X_test_transform_raw.shape)



(13500, 301)
(1500, 301)


##### Hyperparameter search

Instead of exhaustively searching through the parameter grid (slow), or randomly choosing combinations of parameters (missable), we can perform hyperparameter optimization using based on the Bayes Theorem:
https://doi.org/10.11989/JEST.1674-862X.80904120

Using the `hyperopt` package for hyperparameter optimization, comparable mse scores and better performance time has been achieved in contrast to `GridSearchCV`.



In [8]:
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, Trials, space_eval
from sklearn.metrics import mean_squared_error

# Function to be optimized (check the hyperopt doc)
def objective(params):
    rfr = RandomForestRegressor(**params,random_state=42)
    rfr.fit(X_train_transform_raw, y_train_raw)

    raw_y_pred = rfr.predict(X_test_transform)
    mse = mean_squared_error(y_test_raw, raw_y_pred)

    return mse

# Define param grid
space = {
    'n_estimators': hp.choice('n_estimators', range(10, 301)),
    'max_depth': hp.choice('max_depth', range(1, 101)),
    'max_features': hp.choice('max_features', ["sqrt", "log2"])
}

In [14]:
%%time
# Bayesian Optimization using the TPE algorithm `tpe` 
# TPE is sequential-based and balances "explore vs exploit" when 
# searching for hyperparameters: 

trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)

# best hyperparameters and best score
best_params = space_eval(space, best)
best_score = min(trials.losses())

print("Best Parameters:", best_params)
print("Best Score:", best_score)

100%|██████████| 100/100 [47:14<00:00, 28.35s/trial, best loss: 0.003159586913488544]
Best Parameters: {'max_depth': 18, 'max_features': 'sqrt', 'n_estimators': 267}
Best Score: 0.003159586913488544
CPU times: user 47min 4s, sys: 2.93 s, total: 47min 7s
Wall time: 47min 15s


In [14]:
# 100%|██████████| 100/100 [47:14<00:00, 28.35s/trial, best loss: 0.003159586913488544]
# Best Parameters: {'max_depth': 18, 'max_features': 'sqrt', 'n_estimators': 267}
# Best Score: 0.003159586913488544
# CPU times: user 47min 4s, sys: 2.93 s, total: 47min 7s
# Wall time: 47min 15s

In [10]:
### Predicting Parameters

# Saving the best_params without having to rerun the search
best_params = {
    'n_estimators': 267,
    'max_depth': 18,
    'max_features': 'sqrt'
}

rfr_best_raw = RandomForestRegressor(**best_params, random_state=42)
rfr_best_raw.fit(X_train_transform_raw, y_train_raw)

y_pred_raw = rfr_best_raw.predict(X_test_transform_raw)

mse = mean_squared_error(y_test_raw, y_pred_raw)
print(mse)


0.003159586913488544


### Features Model

In [11]:
### Splitting and preparing the raw data

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor


# Split the data into train and test sets
X_train, X_test= train_test_split(features_matrix, test_size=0.1, random_state=42)
y_train_feat, y_test_feat = train_test_split(def_param, test_size=0.1, random_state=42)

# Normalize the data using StandardScaler
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_norm_feat = scaler.fit_transform(X_train)
X_test_norm_feat = scaler.transform(X_test)


print(X_train_norm_feat.shape)
print(X_test_norm_feat.shape)



(13500, 10)
(1500, 10)


In [12]:
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, Trials, space_eval
from sklearn.metrics import mean_squared_error

def objective(params):
    rfr = RandomForestRegressor(**params,random_state=42)
    rfr.fit(X_train_norm_feat, y_train_feat)

    y_pred_feat = rfr.predict(X_test_norm_feat)
    mse = mean_squared_error(y_test_feat, y_pred_feat)

    return mse

# Define param grid
space = {
    'n_estimators': hp.choice('n_estimators', range(1, 500)),
    'max_depth': hp.choice('max_depth', range(1, 150)),
    'max_features': hp.choice('max_features', ["sqrt", "log2"])
}

In [ ]:
%%time
# searching for hyperparameters: 
trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)

# best hyperparameters and best score
best_params = space_eval(space, best)
best_score = min(trials.losses())

print("Best Parameters:", best_params)
print("Best Score:", best_score)

In [ ]:
# 100%|██████████| 100/100 [15:19<00:00,  9.19s/trial, best loss: 0.004580962113038334]
# Best Parameters: {'max_depth': 25, 'max_features': 'sqrt', 'n_estimators': 387}
# Best Score: 0.004580962113038334
# CPU times: user 15min 8s, sys: 8.78 s, total: 15min 16s
# Wall time: 15min 19s

In [14]:
### Predicting Parameters

# Saving the best_params without having to rerun the search
best_params = {
    'n_estimators': 387,
    'max_depth': 25,
    'max_features': 'sqrt'
}

rfr_best_feat = RandomForestRegressor(**best_params, random_state=42)
rfr_best_feat.fit(X_train_norm_feat, y_train_feat)

y_pred_feat = rfr_best_feat.predict(X_test_norm_feat)

mse = mean_squared_error(y_test_feat, y_pred_feat)
print(mse)


0.004618979157521966


##### Model Blending

In [15]:
# Combining the predictions by averaging them 
# There are other methods for model blending/stacking but
# for now, let's take the average.
combined_pred = (y_pred_feat + y_pred_raw) / 2
mse = mean_squared_error(y_test_raw, combined_pred)
print(mse)

0.003547948657251637


### Predicting Based on the Ground Truth

In [ ]:
#pip install spectrapepper

In [ ]:
import spectrapepper as spep
from scipy import signal
from pathlib import Path
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Loading ground truth data into a dataframe
folder_path = './Ground Truth Smoothed'
data = {}
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and file_path.endswith('.csv'):
        with open(file_path, 'r') as file:
            expt_df = pd.read_csv(file_path, header=None, names=['K', 'des_flux'], index_col=None, squeeze = True)
            expt_tds = expt_df.iloc[:, 1].values
            expt_datapts = len(expt_df["des_flux"])
            data[filename] = expt_df    

df = pd.concat(data.values())


expt_des_flux=df['des_flux']
expt_des_flux=expt_des_flux.tolist()
expt_files=int(len(expt_des_flux)/expt_datapts) 
expt_des_flux=np.reshape(expt_des_flux, [expt_files, expt_datapts])

display(len(expt_des_flux[1]))


##### GT Raw predictions

In [17]:

expt_pred_raw = []
for i in range(0,14):
    x_expt = np.array(expt_des_flux[i])
    x_expt = x_expt[0::3].reshape(1, -1)

    X_expt_norm = scaler.fit_transform(x_expt)
    X_expt_norm = scaler.transform(X_expt_norm)
    X_test_transform = np.reshape(X_expt_norm, (X_expt_norm.shape[0], -1))

    y_expt_pred = rfr_best_raw.predict(X_test_transform)

    y_expt_pred = np.reshape(y_expt_pred, (8, 1))
    pred_params = pd.DataFrame({'detrap_en':y_expt_pred[:4,0],
                                'defect_con':y_expt_pred[4:,0]})

    display(f"Ground Truth {i+1}:", pred_params)
    expt_pred_raw.append(y_expt_pred)
    
    y = x_expt.flatten()
    x = np.linspace(300,1200,301)
#    peaks, _ = signal.find_peaks(y, height= 1E17)
#     max_peak = peaks[np.argmax(y[peaks])]
#     x_max = x[max_peak]
#     plt.plot(x, y)
#     plt.plot(x[peaks], y[peaks], 'x')
#     plt.axvline(x=x_max, ls='--', color="k")
#     plt.show()
    peak_pos = np.argmax(x_expt)
    peak_value = y[peak_pos]
    print(peak_pos, peak_value)
    
    ## Exporting as .csv
    src_path  = r'./Predicted/predict_params.csv'
    p = Path(src_path).parent.joinpath(f"raw_pred_params_{i+1}.csv")
    pred_params.to_csv(p, index=False)

'Ground Truth 1:'

,detrap_en,defect_con
0,1.262588,0.007495
1,1.347002,0.008293
2,1.411214,0.006600
3,1.518905,0.004689


152 4.764666666666666e+16


'Ground Truth 2:'

,detrap_en,defect_con
0,1.330487,0.007057
1,1.406452,0.008043
2,1.474680,0.007103
3,1.573686,0.005553


...

,detrap_en,defect_con
0,1.270577,0.007926
1,1.350887,0.007815
2,1.418238,0.007057
3,1.521376,0.005172


156 1.124e+17


'Ground Truth 14:'

,detrap_en,defect_con
0,1.362815,0.007372
1,1.434370,0.007911
2,1.506855,0.006639
3,1.606976,0.005268


126 4.756e+17


##### GT Features redictions

In [18]:
peak_heights = []
fwhm = []
peak_positions = []
area_under_curves = []
stdev = []

for i in expt_des_flux:    
# Peak information
# Peak heights, position, and full width at half max
    peaks, _ = find_peaks(i)
    peak_pos = np.argmax(i)
    peak_height = np.max(i)
    peak_width, _, _, _ = peak_widths(i, [peak_pos], rel_height=0.5) 
        #taking only the first output (widths)
    peak_heights.append(peak_height)
    fwhm.append(peak_width[0])
    peak_positions.append(peak_pos)
# Area under curve
# Calculated using the trapezoidal method
    area_under_curve = auc(range(0,len(i)), i)
    area_under_curves.append(area_under_curve)
# STDEV
    spectra_std = np.std(i)
    stdev.append(spectra_std)
    
pca_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5))])

scaled_expt_des = pca_pipe.fit_transform(expt_des_flux)
result_pca = pca_pipe.fit_transform(scaled_expt_des)


# Formatting them into columns in a df
pca_col = ['PCA Component {}'.format(i+1) for i in range(result_pca.shape[1])]
pca_df = pd.DataFrame(data=result_pca, columns=pca_col)

### Making the features matrix 
features_matrix = pd.DataFrame()
features_matrix['peak_heights'] = peak_heights 
features_matrix['peak_pos'] = peak_positions
features_matrix['fwhm'] = fwhm
features_matrix['std'] = stdev
features_matrix['auc'] = area_under_curves
features_matrix = pd.concat([features_matrix, pca_df],axis=1)
display(features_matrix)

,peak_heights,peak_pos,fwhm,std,auc,PCA Component 1,PCA Component 2,PCA Component 3,PCA Component 4,PCA Component 5
0,4.768667e+16,454,290.162302,1.638158e+16,1.514390e+19,-0.623648,-0.436442,0.372917,-0.881941,2.133504
1,4.199333e+17,548,174.494174,1.384690e+17,7.686186e+19,-0.113093,-1.602602,2.472904,-0.155072,-0.838985
2,1.685333e+17,568,161.047106,5.372150e+16,2.813385e+19,0.193099,1.564766,0.603872,-0.613740,-1.233740
3,3.017333e+17,576,202.024550,1.001815e+17,6.631249e+19,-0.935588,-1.463390,-1.742873,-0.466775,-0.704682
4,2.658667e+17,581,210.426077,8.937861e+16,6.013363e+19,-0.781283,-1.045379,-1.426793,-0.503190,-0.771122
5,2.367333e+17,549,221.706953,8.455735e+16,5.292533e+19,-0.274719,0.448646,0.140682,-0.437093,-1.569831
6,6.932000e+17,380,324.911173,2.459532e+17,1.972324e+20,0.085046,0.019736,-0.095902,1.347879,0.379023
7,3.464667e+17,383,313.634003,1.257978e+17,9.758620e+19,-0.310827,0.497060,0.709534,0.039784,0.047255
8,7.526667e+16,497,209.625626,2.516947e+16,1.604650e+19,0.339345,1.994306,-0.831241,-0.812088,-0.129430
9,2.436000e+17,586,257.656037,8.226860e+16,5.808884e+19,3.394320,-0.923452,-0.470917,-0.210846,0.104772


In [31]:
# Predicting based on features and combining predictions
scaler.fit(X_train)
norm_feat_mat = scaler.transform(features_matrix)
expt_pred_feat = rfr_best_feat.predict(norm_feat_mat)
expt_pred_raw = np.array(expt_pred_raw).reshape(14,8)

combined_expt_pred = ( expt_pred_feat + expt_pred_raw ) / 2

(14, 8)


In [32]:
# Exporting
for num, i in enumerate(expt_pred_feat):
    pred_params = pd.DataFrame({'detrap_en':i[:4],
                                'defect_con':i[4:]})

    src_path  = r'./Predicted/predict_params.csv'
    p = Path(src_path).parent.joinpath(f"feat_pred_params_{num+1}.csv")
    pred_params.to_csv(p, index=False)

for num, i in enumerate(combined_expt_pred):
    pred_params = pd.DataFrame({'detrap_en':i[:4],
                                'defect_con':i[4:]})

    src_path  = r'./Predicted/predict_params.csv'
    p = Path(src_path).parent.joinpath(f"combined_pred_params_{num+1}.csv")
    pred_params.to_csv(p, index=False)
    